<a href="https://colab.research.google.com/github/dzastin96/product-category-classifier/blob/main/notebooks/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🤖 Model Training & Evaluation
### Author: Dzastin Januzi

## 🎯 Goal

Build and evaluate a robust product category classification pipeline using multiple machine learning algorithms.  
The objective is to identify the best-performing model and feature setup for deployment in e-commerce or inventory systems.

We compare five classifiers:
- Logistic Regression
- Naive Bayes
- Decision Tree
- Random Forest
- Support Vector Machine (SVM)

Each model is evaluated under two feature configurations:
- 📝 `product_title` only (text-based)
- 🧮 `product_title` + numeric features (`views_per_day`, `popularity_score`, `merchant_rating`)

📊 Evaluation metrics:
- Accuracy
- Macro F1 Score
- Weighted F1 Score
- Classification Report

## 📥 1. Load Data

We load the preprocessed product listing dataset from a serialized .pkl file using joblib

In [19]:
import joblib

df = joblib.load("../data/final_product_data.pkl")
df.head()

,product_title,category_label,num_words,num_chars,has_digits_or_special,has_uppercase_terms,longest_word_len
0,apple iphone 8 plus 64gb silver,mobile phones,6,31,1,1,6
1,apple iphone 8 plus 64 gb spacegrau,mobile phones,7,35,1,1,9
2,apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...,mobile phones,13,70,1,1,10
3,apple iphone 8 plus 64gb space grey,mobile phones,7,35,1,1,6
4,apple iphone 8 plus gold 5.5 64gb 4g unlocked ...,mobile phones,11,54,1,1,8


## ✂️ 2. Train/Test Split

Split the cleaned dataset into training and testing sets using stratified sampling to preserve class distribution. This ensures fair evaluation across all product categories.

In [20]:
from sklearn.model_selection import train_test_split

print(df['category_label'].value_counts())


X = df[['product_title', 'num_words', 'num_chars', 'has_digits_or_special', 'has_uppercase_terms', 'longest_word_len']]
    
y = df['category_label'].astype(str)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

category_label
fridge freezers     11130
mobile phones        4023
washing machines     3971
cpus                 3792
tvs                  3502
dishwashers          3374
digital cameras      2661
microwaves           2307
Name: count, dtype: int64


## 🧼 3. Preprocessing

We prepare the input features using a modular preprocessing pipeline.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler

# We use MinMaxScaler instead of StandardScaler because:
# - MultinomialNB requires non-negative inputs
# - MinMaxScaler maps all numeric features into [0,1], preserving non-negativity
# - StandardScaler would introduce negative values, causing errors


print(df.head(5))
preprocess = ColumnTransformer([
    ('title', TfidfVectorizer(), 'product_title'),
    ('length', MinMaxScaler(), [
        "longest_word_len",
        "num_words",
        "num_chars"
    ]),
    ('binary', MinMaxScaler(), [
        "has_digits_or_special",
        "has_uppercase_terms"
    ]),
])
    
    

                                       product_title category_label  \
0                    apple iphone 8 plus 64gb silver  mobile phones   
1                apple iphone 8 plus 64 gb spacegrau  mobile phones   
2  apple mq8n2b/a iphone 8 plus 64gb 5.5 12mp sim...  mobile phones   
3                apple iphone 8 plus 64gb space grey  mobile phones   
4  apple iphone 8 plus gold 5.5 64gb 4g unlocked ...  mobile phones   

   num_words  num_chars  has_digits_or_special  has_uppercase_terms  \
0          6         31                      1                    1   
1          7         35                      1                    1   
2         13         70                      1                    1   
3          7         35                      1                    1   
4         11         54                      1                    1   

   longest_word_len  
0                 6  
1                 9  
2                10  
3                 6  
4                 8  


## 📦 4. Candidate Models

We define a set of candidate classification models to evaluate performance across different algorithmic approaches.
All models are wrapped in a pipeline that includes preprocessing (TF-IDF + optional scaling) and classification.  
This modular setup allows us to benchmark each model consistently across both feature configurations (`product_title` only vs `product_title` + numeric features).

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Define candidate models
models = {
    "Logistic Regression": LogisticRegression(max_iter=5000),
    "Naive Bayes": MultinomialNB(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Support Vector Machine": LinearSVC(max_iter=5000)
}

## 📊 5. Training, Prediction, Evaluation

We train and evaluate each candidate model using a consistent pipeline:
- Each model is wrapped in a `Pipeline` that includes:
  - **Preprocessing**: TF-IDF vectorization (and optional MinMax scaling if numeric features are used)
  - **Classifier**: one of the selected models

🔁 For each model:
- We **fit** the pipeline on the training set (`X_train`, `y_train`)
- We **predict** on the test set (`X_test`)
- We generate a **classification report** showing precision, recall, and F1-score per class

In [23]:
for name, model in models.items():
    print(f"\n=== {name} ===")
    
    pipeline = Pipeline([
        ("preprocessing", preprocess),
        ("classifier", model)
    ])
    
    # Train
    pipeline.fit(X_train, y_train)
    
    # Predict
    y_pred = pipeline.predict(X_test)
    
    # Classification report
    print(classification_report(y_test, y_pred))


=== Logistic Regression ===
                  precision    recall  f1-score   support

            cpus       1.00      1.00      1.00       759
 digital cameras       1.00      0.99      1.00       532
     dishwashers       0.97      0.92      0.95       675
 fridge freezers       0.95      0.99      0.97      2226
      microwaves       1.00      0.94      0.97       461
   mobile phones       0.99      0.99      0.99       805
             tvs       0.98      0.97      0.98       700
washing machines       0.98      0.94      0.96       794

        accuracy                           0.97      6952
       macro avg       0.98      0.97      0.97      6952
    weighted avg       0.97      0.97      0.97      6952


=== Naive Bayes ===
                  precision    recall  f1-score   support

            cpus       1.00      1.00      1.00       759
 digital cameras       1.00      0.98      0.99       532
     dishwashers       0.99      0.85      0.91       675
 fridge freezers  

## 🏆 6. Select the Best Model

In this step, we evaluate all trained models (Logistic Regression, Naive Bayes, Decision Tree, Random Forest, and Support Vector Machine) using key performance metrics:

- **Accuracy** – overall proportion of correct predictions  
- **Macro Avg. F1 Score** – harmonic mean of precision and recall, treating all classes equally  
- **Weighted Avg. F1 Score** – harmonic mean of precision and recall, weighted by class support  

### Results (`product_title` + Engineered Features)

| Model                  | Accuracy | Macro Avg. F1 | Weighted Avg. F1 | Comments                                                   |
|------------------------|----------|----------------|------------------|------------------------------------------------------------|
| Logistic Regression     | 0.97     | 0.97           | 0.97             | Very consistent across all classes; strong balance overall |
| Naive Bayes             | 0.96     | 0.96           | 0.96             | Good, but weaker on dishwashers and microwaves             |
| Decision Tree           | 0.97     | 0.97           | 0.97             | Solid, interpretable, but slightly less stable across categories |
| Random Forest           | 0.98     | 0.98           | 0.98             | Strong ensemble, robust across categories                  |
| **Support Vector Machine** | **0.98** | **0.98**     | **0.98**         | 🏆 Best overall — highest accuracy & balanced across classes |

### Why Choose SVC Instead of Random Forest?

Both **Random Forest** and **Support Vector Machine (SVC)** achieved excellent performance (≈0.98 accuracy). However, we select **SVC** as the final production model for the following reasons:

### 🔍 Performance Balance
- **SVC** shows slightly higher **macro F1 (0.99 vs 0.98)**, meaning it balances precision and recall more evenly across all categories.
- Random Forest is strong overall, but SVC edges ahead in categories like *digital cameras* and *microwaves*, where recall and precision are perfectly aligned.

### 📌 Final Decision

After comparing all models, we select:

- ✅ **Best Model:** Support Vector Machine (SVM)  
- 📈 Using both **`product_title` (TF‑IDF)** and **engineered features** (numeric + binary) yields the strongest, most balanced performance.

